In [3]:
import pandas as pd
import requests
import json
import re
import time
from tqdm import tqdm


df = pd.read_csv("debate_results_llama3.csv", encoding="utf-8")
print(f"✅ File loaded successfully! Total samples: {len(df)}")
print("Columns:", df.columns.tolist())


if "combined_dialogue" not in df.columns:
    dialogue_cols = [c for c in df.columns if c.startswith("round") or "reflection" in c]
    df["combined_dialogue"] = df[dialogue_cols].astype(str).apply(lambda x: "\n".join(x), axis=1)
    print("✅ Combined dialogues successfully.")
else:
    print("Using existing 'combined_dialogue' column.")


def get_qwen_score(dialogue_text, max_retries=3):

    prompt = f"""
You are an impartial evaluator. 
Rate how much the two speakers in the following dialogue agree with each other (1–5).
1 = totally disagree, 5 = totally agree.

Dialogue:
{dialogue_text}

Please output ONLY the number (1–5).
"""

    for attempt in range(max_retries):
        try:
            response = requests.post(
                "http://localhost:11434/api/generate",
                json={"model": "qwen2.5:7b", "prompt": prompt},
                stream=False,
                timeout=120
            )

            # 嘗試解析 Ollama stream JSON
            text = ""
            for line in response.text.splitlines():
                try:
                    data = json.loads(line)
                    if "response" in data:
                        text += data["response"]
                except json.JSONDecodeError:
                    continue

            # 嘗試從模型回應中提取分數（1–5）
            match = re.search(r"[1-5]", text)
            if match:
                return int(match.group())

        except Exception as e:
            print(f"⚠️ Attempt {attempt+1} failed: {e}")
            time.sleep(3)

    return None



scores = []
for dialogue in tqdm(df["combined_dialogue"], desc="🧠 Evaluating with Qwen2.5-7B..."):
    score = get_qwen_score(dialogue)
    scores.append(score)

df["agreement_score"] = scores


output_file = "llama3_dialogue_qwen_scores.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"✅ Done! Results saved as {output_file}")

✅ File loaded successfully! Total samples: 90
Columns: ['topic_id', 'pair_type', 'topic_statement', 'A_occupation', 'A_region', 'A_pref', 'B_occupation', 'B_region', 'B_pref', 'round1_A', 'round1_B', 'round2_A', 'round2_B', 'round3_A', 'round3_B', 'reflection_A', 'reflection_B']
✅ Combined dialogues successfully.


🧠 Evaluating with Qwen2.5-7B...: 100%|█████████| 90/90 [05:37<00:00,  3.75s/it]

✅ Done! Results saved as debate_results_qwen_local.csv
